In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    classification_report
)

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline


In [2]:
# Cargar datos ya preparados
churn_features = pd.read_csv("../data/churn_features.csv")


In [ ]:
y = churn_features["churn"]
X = churn_features.drop(columns=["customer_id", "churn", "last_purchase_date"])
